In [245]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.metrics import f1_score, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, GroupKFold
import time
import pickle
from datetime import date as dt
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from eli5.sklearn import PermutationImportance
from eli5 import show_weights

In [237]:
DATASET_PATH_TRAIN = './data_train.csv'
train = pd.read_csv(DATASET_PATH_TRAIN, sep=',')
train['vas_id'] = train['vas_id'].astype('int8')
train['target'] = train['target'].astype('int8')
train['vas_id'] = train['vas_id'].astype('object')
train['buy_time']=train['buy_time'].map(lambda x: dt.fromtimestamp(x))
train['buy_time'] = pd.to_datetime(train['buy_time']) 
y = train['target']
X = train.drop(['Unnamed: 0', 'target'],axis=1)
categorical = train.dtypes[train.dtypes == "object"].index.tolist()
def catboost_cross_validation(params, X, y, cv, categorical= None):
    estimators= []
    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y, groups = X["id"])):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
        model = cb.CatBoostClassifier(**params)
        model.fit(
             x_train, y_train, categorical,
             eval_set=[(x_train, y_train), (x_valid, y_valid)]               
            )
        estimators.append(model)
    return estimators
scale = int(y[y == 0].shape[0] / y[y == 1].shape[0])
    
cb_params = {'silent': True, 
            'scale_pos_weight': scale,
            
    }
cv = GroupKFold(n_splits=3)
estimators = catboost_cross_validation(
params=cb_params, X=X, y=y, cv=cv, categorical=categorical)

In [238]:
def catmodel():
    DATASET_PATH_TEST = './data_test.csv'    
    test = pd.read_csv(DATASET_PATH_TEST, sep=',')    
    test['vas_id'] = test['vas_id'].astype('int8')
    test['vas_id'] = test['vas_id'].astype('object')
    test['buy_time']=test['buy_time'].map(lambda x: dt.fromtimestamp(x))
    test['buy_time'] = pd.to_datetime(test['buy_time'])
    test=test.drop(['Unnamed: 0'],axis=1)
    categorical = test.dtypes[train.dtypes == "object"].index.tolist()
    y_pred=np.zeros(test.shape[0])
    for estimator in estimators:
        y_pred += estimator.predict_proba(test)[:, 1]
    prediction = y_pred/len(estimators)
    prediction = pd.Series(prediction)
    prediction =np.where(prediction>0.85, 1, 0)
    test['target'] = prediction
    test.to_csv('./answers_test.csv', sep=",", index=False)

In [ ]:
catmodel()

In [247]:
with open("model.pkl", "wb") as f:
    pickle.dump(catmodel, f)